<a href="https://colab.research.google.com/github/andreidore/aicrowd/blob/master/blitz_3/snake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm

In [2]:
import os
import tensorflow as tf


In [3]:
print(tf.test.is_gpu_available("GPU"))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


In [ ]:
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/snake/v0.1/train.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/snake/v0.1/val.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/snake/v0.1/test.zip

!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/snake/v0.1/sample_submission.csv


!mkdir data
!mkdir data/test
!mkdir data/train
!mkdir data/val

!unzip -q train.zip
!unzip -q test.zip
!unzip -q val.zip


!mv content/data/train/* data/train
!mv content/data/val/* data/val
!mv content/data/test/* data/test


!rm -rf content/


!mkdir models